In [4]:
#!pip install surprise

In [5]:
#load pacakges
#virtual env tensorflow built based on:
#https://geneashis.medium.com/tensorflow-part-1-getting-started-with-tensorflow-import-tensorflow-as-tf-29008f7f649a
#https://janakiev.com/blog/jupyter-virtual-envs/
import os
import numpy as np
import pandas as pd
#import geopandas as gpd
import csv
#import pandasql as ps
import matplotlib.pyplot as plt
#import shapefile as shp
import seaborn as sns
import random
import time
from sklearn.model_selection import train_test_split
#import pysal as ps
import scipy
#import tensorflow as tf

import surprise
import itertools

In [6]:
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor
from surprise import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import BaselineOnly, CoClustering
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import Dataset

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
#load data

#save matrice
#np.savetxt('test_matrice_reverse.csv', test_matrice_reverse, delimiter=",")
#np.savetxt('Zero_index.csv', Zero_index, delimiter=",")

#read the test matrice and data
test_matrice_reverse = pd.read_csv('test_matrice_reverse.csv', header=None).to_numpy()
Zero_index = pd.read_csv('Zero_index.csv', header=None).to_numpy()

In [39]:
#PCA
#scale the data
test_standard = StandardScaler().fit_transform(test_matrice_reverse[:, 1:])

In [60]:
a = np.array([[1,2], [6,16]])

pca = PCA(n_components=2)

test = pca.fit_transform(a)

In [61]:
principalDf = pd.DataFrame(data = test,
                           columns = ['principal component 1', 
                                      'principal component 2'])

principalDf

,principal component 1,principal component 2
0,-7.433034,2.112317e-16
1,7.433034,2.112317e-16


In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
#targets = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = finalDf['target'] == target
    ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
               , finalDf.loc[indicesToKeep, 'principal component 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [13]:
test_matrice_reverse[:, 1:].shape

(7767, 3456)

In [9]:
#example
'''a = np.ones((2,2))
a[0][1] = 2
a[1][1] = 6
a[1][0] = 3
print(a)
print(a.flatten())
print(a.flatten('F'))'''

#get ids and st ids
with open('user_tsinfo.txt') as f:
    infos = f.readlines()
    
user_ids = infos[1].split(',')
item_ids = infos[0].split(',')
user_ids = user_ids[:-1]
item_ids = item_ids[:-1]

n_item = len(item_ids)
n_user = len(user_ids)

test_dt_obs = test_matrice_reverse[:,1:].flatten('F')
test_dt_users = np.tile(user_ids, n_item)
test_dt_items = np.repeat(item_ids, n_user)

test_dt = pd.DataFrame(test_dt_users, columns=['userID'])
test_dt['item'] = test_dt_items
test_dt['obs'] = test_dt_obs

del(test_dt_obs)
del(test_dt_users)
del(test_dt_items)

reader = Reader(rating_scale=(0, np.max(test_matrice_reverse[:, 1:])))
dt_test_ = Dataset.load_from_df(test_dt[['userID', 'item', 'obs']], reader)

#code reference
#https://towardsdatascience.com/building-and-testing-recommender-systems-with-surprise-step-by-step-d4ba702ef80b
#surprise cannot handle big data very well

#discussion of PCA


In [11]:
benchmark = []

k = 2
for algorithm in [SVD(n_factors=k), SVDpp(n_factors=k), SlopeOne(), NMF(n_factors=k), 
                  NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), 
                  KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, dt_test_, measures=['RMSE'], cv=2, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
test = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

ZeroDivisionError: float division

In [12]:
test

NameError: name 'test' is not defined

In [ ]:
test.to_csv('mf_test.csv')